In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from scipy.stats import randint
import pickle
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, KFold, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score,accuracy_score,confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, classification_report 
from xgboost import XGBClassifier
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample,shuffle

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
import time
from sklearn.metrics import accuracy_score
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from lightgbm import LGBMClassifier

/home/haojie/anaconda3/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
train = pd.read_csv('data/train_clean_v3.csv')
valid = pd.read_csv('data/tmp_clean_v3.csv')

# 过采样
# train_big = train[train['Response']==0]
# train_small = train[train['Response']==1]
# train_small_up = resample(train_small,replace=True,n_samples=len(train_big),random_state = 0)
# train_balance = pd.concat([train_big, train_small_up])
# train_balance = shuffle(train_balance)
# train_balance.Response.value_counts()

train_target = train['Response']
train.drop(['Response'], axis=1, inplace=True)
valid_target = valid['Response']
valid.drop(['Response'], axis=1, inplace=True)

train_X, test_X, train_Y, test_Y = train_test_split(train, train_target, test_size=0.1)

In [4]:
# 升维
poly = PolynomialFeatures(degree = 2, include_bias=False)
train_X_plus = poly.fit_transform(train_X)
test_X_plus = poly.fit_transform(test_X)
valid_plus = poly.fit_transform(valid)

In [5]:
# 特征选择
time_start = time.time()
# clf=XGBClassifier(n_estimators = 300, max_depth = 13, gamma = 2.3408807913619945, reg_lambda = 0.3770436657913232,
#                             reg_alpha = 40.0, min_child_weight=7.0,colsample_bytree = 0.5786479102658189 ,random_state = 0).fit(train_X_plus, train_Y)
clf = GradientBoostingClassifier().fit(train_X_plus, train_Y)
# print("feature importance after trainning:", clf.feature_importances_)
print("totoal trainning leghth:", time.time() - time_start)

sel_model = SelectFromModel(clf, prefit=True, threshold=-np.inf, max_features=30)
train_X_2 = sel_model.transform(train_X_plus)
print("新的特征数量:", train_X_2.shape[1])

totoal trainning leghth: 329.2567603588104
新的特征数量: 30


In [6]:
test_X_2 = sel_model.transform(test_X_plus)
valid_2 = sel_model.transform(valid_plus)

In [7]:
## XGBOOST
# space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
#         'gamma': hp.uniform ('gamma', 1,9),
#         'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
#         'reg_lambda' : hp.uniform('reg_lambda', 0,1),
#         'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
#         'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
#         'n_estimators': 300,
#         'seed': 0
#     }

## LGBM
from hyperopt import fmin, tpe, hp, partial

# 自定义hyperopt的参数空间
params_space = {
    'max_depth': hp.randint('max_depth', 12),
    'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
    # 'n_estimators': hp.quniform("n_estimators", 100, 500, 50),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 3, 15, 2)
}

In [10]:
# f = open('Hypeparam_tuning.txt', 'w')
f = open('Hypeparam_tuning_lgbm.txt', 'w')

In [13]:
'''
XGBOOST
'''
# def objective(space):
#     clf=xgb.XGBClassifier(
#                     n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
#                     reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
#                     colsample_bytree=int(space['colsample_bytree']))
    
#     evaluation = [(train_X_2, train_Y), (test_X_2, test_Y)]
    
#     clf.fit(train_X_2, train_Y,
#             eval_set=evaluation, eval_metric="auc",
#             early_stopping_rounds=10,verbose=False)

#     yscore = clf.predict_proba(test_X_2)[:,1]
#     fpr, tpr, _ = roc_curve(test_Y, yscore)
#     score = auc(fpr, tpr)
#     print("Score:", score)
#     print ("Hypeparams:", space, "SCORE:", score, file=f)
#     return {'loss': 1-score, 'status': STATUS_OK }


'''
LGBM
'''
def objective(params):
    clf = LGBMClassifier(
        n_jobs = -1,
        max_depth=int(params['max_depth']) + 3,
        learning_rate=params['learning_rate'],
        n_estimators=400,
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        min_data_in_leaf=int(params["min_data_in_leaf"])  # 最小叶子节点样本数
    )
    # clf.set_params(params=params)
    evaluation = [(train_X_2, train_Y), (test_X_2, test_Y)]
    
    clf.fit(train_X_2, train_Y,
            eval_set=evaluation, eval_metric="auc", verbose=False)

    yscore = clf.predict_proba(test_X_2)[:,1]
    fpr, tpr, _ = roc_curve(test_Y, yscore)
    score = auc(fpr, tpr)
    print("Score:", score)
    print ("Hypeparams:", params, "SCORE:", score, file=f)
    return {'loss': 1-score, 'status': STATUS_OK }

In [14]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = params_space,
                        algo = tpe.suggest,
                        max_evals = 200,
                        trials = trials)

578716362149413
[LightGBM] [Warning] 
min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
Score:
0.8581107592654267
[LightGBM] [Warning] 
min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Score:
0.8555211340717004
[LightGBM] [Warning] 
min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
Score:
0.8581584231256295
[LightGBM] [Warning] 
min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
Score:
0.8581155564871186
[LightGBM] [Warning] 
min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] 
Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Score:
0.8553972606074658
[LightGBM] [Warning] 
min_data_in_leaf 

In [55]:
# f2 = open("Best_params_for_XGBoost.txt", 'w')
# print(best_hyperparams, file=f2)
# f.close()
# f2.close()  

f2 = open("Best_params_for_LGBM.txt", 'w')
print(best_hyperparams, file=f2)
f.close()
f2.close()  

In [15]:
best_hyperparams

{'colsample_bytree': 0.9852486122992379,
 'learning_rate': 0.12528230409524013,
 'max_depth': 0,
 'min_data_in_leaf': 4.0,
 'subsample': 0.7637764826473344}

In [60]:
clf2 = LGBMClassifier(colsample_bytree=0.9852486122992379, learning_rate=0.12528230409524013,min_data_in_leaf=4, subsample=0.7637764826473344, n_estimators=33, n_jobs=-1)
clf2.fit(train_X_2, train_Y)

[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4


LGBMClassifier(colsample_bytree=0.9852486122992379,
               learning_rate=0.12528230409524013, min_data_in_leaf=4,
               n_estimators=33, subsample=0.7637764826473344)

In [61]:
yscore = clf2.predict_proba(valid_2)[:,1]
fpr, tpr, _ = roc_curve(valid_target, yscore)
score = auc(fpr, tpr)
print("Score:", score)

Score: 0.859125543028784


In [62]:
yscore = clf2.predict_proba(test_X_2)[:,1]
fpr, tpr, _ = roc_curve(test_Y, yscore)
score = auc(fpr, tpr)
print("Score:", score)

Score: 0.8577315057802528


In [63]:
yscore = clf2.predict_proba(train_X_2)[:,1]
fpr, tpr, _ = roc_curve(train_Y, yscore)
score = auc(fpr, tpr)
print("Score:", score)

Score: 0.8611072196284626


In [117]:
# clf = LGBMClassifier(colsample_bytree=0.917149292140063, learning_rate=0.038244008834829285, max_depth=5, min_data_in_leaf=14, subsample=0.9007129277879421, n_estimators=300, n_jobs=-1)
clf = LGBMClassifier(n_jobs=-1, colsample_bytree=0.917149292140063, learning_rate=0.038244008834829285, n_estimators=888, max_depth=7)
clf.fit(train_X_2, train_Y)

LGBMClassifier(colsample_bytree=0.917149292140063,
               learning_rate=0.038244008834829285, n_estimators=666)

In [118]:
yscore = clf.predict_proba(train_X_2)[:,1]
fpr, tpr, _ = roc_curve(train_Y, yscore)
score = auc(fpr, tpr)
print("Score:", score)

Score: 0.877493952901573


In [119]:
yscore = clf.predict_proba(test_X_2)[:,1]
fpr, tpr, _ = roc_curve(test_Y, yscore)
score = auc(fpr, tpr)
print("Score:", score)

Score: 0.8575615708997317


In [120]:
yscore = clf.predict_proba(valid_2)[:,1]
fpr, tpr, _ = roc_curve(valid_target, yscore)
score = auc(fpr, tpr)
print("Score:", score)

Score: 0.8591125251346101
